In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16

print("Import OK")

Using TensorFlow backend.


Import OK


In [14]:
model = VGG16(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

x = model.output
x = GlobalAveragePooling2D()(x)
#x = Dense(1024, activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
#x = Dense(1024, activation='relu')(x) #dense layer 2
x = Dense(512, activation='relu')(x) #dense layer 3
#x = Dense(256, activation='relu')(x) #dense layer 4
#x = Dropout(0.25)(x)
x = Dense(128, activation='relu')(x) #dense layer 5
#x = Dense(64, activation='relu')(x) #dense layer 6
#x = Dropout(0.3)(x)
#x = Dense(32, activation='relu')(x)
preds = Dense(13, activation='softmax')(x) #final layer with softmax activation

model2 = Model(inputs=model.input, outputs=preds)
for layer in model2.layers[:20]:
    layer.trainable=False
for layer in model2.layers[20:]:
    layer.trainable=True
    
model2.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0   

In [15]:
from keras.applications.vgg16 import preprocess_input, decode_predictions
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/train/', # this is where you specify the path to the main data folder
                                                 target_size=(256, 256),
                                                 #color_mode='grayscale',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

Found 2819 images belonging to 13 classes.


In [16]:
model2.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model2.fit_generator(generator=train_generator, steps_per_epoch=step_size_train, epochs=10)

Epoch 1/10
88/88 [==============================] - 9s 105ms/step - loss: 0.6303 - accuracy: 0.8070
Epoch 2/10
88/88 [==============================] - 9s 103ms/step - loss: 0.1880 - accuracy: 0.9325
Epoch 3/10
88/88 [==============================] - 9s 102ms/step - loss: 0.1661 - accuracy: 0.9442
Epoch 4/10
88/88 [==============================] - 9s 102ms/step - loss: 0.0956 - accuracy: 0.9709
Epoch 5/10
88/88 [==============================] - 9s 102ms/step - loss: 0.0433 - accuracy: 0.9867
Epoch 6/10
88/88 [==============================] - 9s 102ms/step - loss: 0.0246 - accuracy: 0.9939
Epoch 7/10
88/88 [==============================] - 9s 103ms/step - loss: 0.0143 - accuracy: 0.9982
Epoch 8/10
88/88 [==============================] - 9s 102ms/step - loss: 0.0129 - accuracy: 0.9978
Epoch 9/10
88/88 [==============================] - 9s 101ms/step - loss: 0.0067 - accuracy: 0.9982
Epoch 10/10
88/88 [==============================] - 9s 102ms/step - loss: 0.0038 - accuracy: 1.0000

In [17]:
import cv2

test_data_name = []
test_data = []

for dirname, _, filenames in os.walk('/kaggle/input/cs-ioc5008-hw1/dataset/dataset/test'):
    for filename in filenames:
        test_data_name.append(filename)
        image = cv2.imread(os.path.join(dirname, filename))
        test_data.append(image)
        
print("Number of test data =", len(test_data))

Number of test data = 1040


In [18]:
def processing(image_data):
    #img = [data[123], data[1054], data[156]]
    #print("Original size:", img[1].shape)
    #print(training_label[1054])
    #original = img[1]
    #display_one(original)
    
    height = 256
    width = 256
    dim = (width, height)
    res_img = []
    
    for i in range(len(image_data)):
        res = cv2.resize(image_data[i], dim, interpolation=cv2.INTER_LINEAR)
        res_img.append(res)
    
    return res_img

test_data_reshape = processing(test_data)

print(test_data_reshape[0].shape)

(256, 256, 3)


In [20]:
for i in range(len(test_data_reshape)):
    test_data_reshape[i] = preprocess_input(test_data_reshape[i])

In [21]:
#for i in range(len(test_data_reshape)):
#    test_data_reshape[i] = (test_data_reshape[i] - 128.0) / 128.0
    
for i in range(len(test_data_reshape)):
    test_data_reshape[i] = test_data_reshape[i].reshape(-1, 256, 256, 3)
    
scene_detail = ["bedroom", "coast", "forest", "highway", "insidecity", "kitchen", "livingroom", "mountain", "office", "opencountry", "street", "suburb", "tallbuilding"]

print("id,label")

for i in range(len(test_data_reshape)):
    name_arr = test_data_name[i].split('.')
    name = name_arr[0]
    answer = model2.predict(test_data_reshape[i])
    print(name+","+scene_detail[np.argmax(answer)])


id,label
image_0115,street
image_0132,kitchen
image_0661,office
image_0378,tallbuilding
image_0917,livingroom
image_0809,kitchen
image_1028,insidecity
image_0909,livingroom
image_0879,opencountry
image_0562,insidecity
image_0837,forest
image_0309,suburb
image_0902,suburb
image_0911,opencountry
image_0495,opencountry
image_0592,forest
image_0687,livingroom
image_0964,bedroom
image_0049,tallbuilding
image_0432,coast
image_0839,opencountry
image_0250,suburb
image_0951,mountain
image_0183,livingroom
image_0575,bedroom
image_0418,livingroom
image_0605,street
image_0932,opencountry
image_0069,bedroom
image_0949,forest
image_0233,office
image_0999,mountain
image_0301,opencountry
image_0579,tallbuilding
image_0195,insidecity
image_0186,kitchen
image_0739,forest
image_0368,bedroom
image_0238,livingroom
image_0280,forest
image_0460,forest
image_0332,office
image_0865,bedroom
image_0395,coast
image_0512,opencountry
image_0487,forest
image_0201,opencountry
image_0538,suburb
image_0674,opencountry


image_0969,suburb
image_0420,street
image_0633,opencountry
image_0810,livingroom
image_0801,mountain
image_0427,insidecity
image_1027,mountain
image_0434,suburb
image_0329,mountain
image_0521,opencountry
image_0082,livingroom
image_0279,forest
image_0880,highway
image_0451,tallbuilding
image_0327,forest
image_0822,mountain
image_0798,insidecity
image_0685,office
image_0757,opencountry
image_1012,office
image_0610,highway
image_0728,forest
image_0255,highway
image_0656,insidecity
image_0411,opencountry
image_0874,kitchen
image_0659,opencountry
image_0285,street
image_0736,office
image_0519,office
image_0530,livingroom
image_0534,opencountry
image_0208,suburb
image_0282,forest
image_0884,suburb
image_0391,bedroom
image_0345,livingroom
image_0975,insidecity
image_0402,forest
image_0221,opencountry
image_0954,mountain
image_0825,street
image_1026,suburb
image_0886,office
image_0105,mountain
image_0303,kitchen
image_0046,coast
image_0852,mountain
image_0315,suburb
image_0379,street
image_03

image_0650,mountain
image_0137,livingroom
image_0915,forest
image_0321,opencountry
image_0856,kitchen
image_1004,insidecity
image_0414,mountain
image_0073,livingroom
image_0820,mountain
image_0007,forest
image_0251,suburb
image_0982,forest
image_0767,street
image_0382,bedroom
image_0022,highway
image_1011,mountain
image_0778,mountain
image_0247,mountain
image_0230,office
image_0172,street
image_0541,tallbuilding
image_0676,suburb
image_0036,street
image_0364,forest
image_0380,opencountry
image_0946,highway
image_0079,opencountry
image_0019,insidecity
image_0737,insidecity
image_0498,insidecity
image_0080,mountain
image_0066,opencountry
image_0875,coast
image_0574,livingroom
image_0333,insidecity
image_0089,street
image_0895,office
image_0923,kitchen
image_0776,suburb
image_0497,livingroom
image_0731,street
image_1002,forest
image_0729,opencountry
image_0426,street
image_0113,bedroom
image_0763,suburb
image_0335,highway
image_0760,office
image_0108,bedroom
image_0298,highway
image_0401,